Updating the SED files for MEATS to newer MAST standards

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os
import glob
from astropy.table import Table
from astropy.io import ascii
from astropy.convolution import convolve, Box1DKernel
from scipy.interpolate import interp1d

import astropy.units as u
from astropy import time, coordinates as coord
import astropy.constants as const
from astropy.modeling import models, fitting
from astropy.coordinates import SkyCoord
from matplotlib.gridspec import GridSpec
from datetime import datetime
from astroquery.simbad import Simbad

#matplotlib set up
%matplotlib inline
from matplotlib import rcParams
rcParams["figure.figsize"] = (14, 5)
rcParams["font.size"] = 20

In [2]:
oldv = 1
version = 25


    STIS spectra

DOI = ‘10.17909/T9DG6F’

EQUINOX =’2000.2’ or RADESYS = 'ICRS'

HLSPID =’MUSCLES’

HLSPVER=’V1’

LICENSE =’CC BY 4.0’

LICENSURL= 'https://creativecommons.org/licenses/by/4.0/'

TIMESYS: Character string code (see FITS Standard) for the time scale of time-related keywords. Ex: 'UTC'

 

    Model spectra

The same set of metadata is required as STIS data except for EQUINOX/RADESYS and TIMESYS.

 

    SED data 

The same set of metadata is required as STIS data except for OBSERVAT.

MJD-BEG and MJD-END in the primary header. (* see below)

 

    XMM data

The same set of metadata is required as STIS data.

RA_TARG and DEC_TARG values are missing. Please include the correct RA/Dec values.

In [3]:
stars = os.listdir('draft_hlsp/')
# print(stars)
stars = np.array(stars)
mask = stars != '.ipynb_checkpoints'
# print(mask)
stars = stars[mask]
print(stars)
print(len(stars))

['toi-836' 'hd120411' 'hd80606' 'toi-178' 'lhs_1140' 'wasp-166' 'k2-18'
 'gj_4102' 'kepler-51' 'k2-141' 'kappa1_ceti' 'wasp-63' 'toi-402'
 'toi-776' 'kap1cet' 'toi-260' 'gj_341' 'tau_ceti' 'toi-421' 'gj_367'
 'toi-134' 'wasp-121']
22


In [4]:
star_list = Simbad.query_objects(stars)
star_list

/home/david/anaconda3/envs/stenv/lib/python3.11/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 17 raised an error (recorded in the `errors` attribute of the result table): 'kap1cet': No known catalog could be found
  warnings.warn("Warning: The script line number %i raised "


MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,int32
CD-23 12010,15 00 19.3983,-24 27 14.693,14,14,0.014,0.013,90,A,O,2020yCat.1350....0G,1
HD 120411,13 50 06.2797,-40 50 08.881,14,14,0.014,0.018,90,A,O,2020yCat.1350....0G,2
HD 80606,09 22 37.5768,+50 36 13.435,14,14,0.013,0.011,90,A,O,2020yCat.1350....0G,3
WT 2353,00 29 12.3029,-30 27 13.466,14,14,0.023,0.016,90,A,O,2020yCat.1350....0G,4
G 268-38,00 44 59.3309,-15 16 17.542,14,14,0.036,0.022,90,A,O,2020yCat.1350....0G,5
BD-20 2976,09 39 30.0861,-20 58 56.881,14,14,0.013,0.011,90,A,O,2020yCat.1350....0G,6
K2-18,11 30 14.5177,+07 35 18.255,14,14,0.022,0.020,90,A,O,2020yCat.1350....0G,7
L 22-69,19 20 54.3761,-82 33 16.167,14,14,0.016,0.017,90,A,O,2020yCat.1350....0G,8


In [5]:

newpath = 'fixed_hlsp/'
for star in stars:
    print(star)
    newdir = '{}{}'.format(newpath, star)
    if not os.path.exists(newdir):
       os.makedirs(newdir)
    
    oldv = 1
    if star == 'wasp-63':
        oldv = 2
    starpath = 'draft_hlsp/{}/'.format(star)
    specs = np.hstack((glob.glob('{}*stis*.fits'.format(starpath)),glob.glob('{}*cos*.fits'.format(starpath))))
    mods = glob.glob('{}*mod*.fits'.format(starpath))
    seds = glob.glob('{}*multi*.fits'.format(starpath))
    xrays = np.hstack((glob.glob('{}*cxo*.fits'.format(starpath)),glob.glob('{}*xmm*.fits'.format(starpath))))

    # print(specs)
    # print(mods)
    # print(seds)
    # print(xrays)
    
    
    starts = []
    ends = []

    for spec in specs:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        ra, dec = hdr['RA_TARG'], hdr['DEC_TARG']
        starts.append(hdr['EXPSTART'])
        ends.append(hdr['EXPEND'])
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    for spec in mods:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    for spec in xrays:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        starts.append(hdr['EXPSTART'])
        ends.append(hdr['EXPEND'])
        hdr.append(('RA_TARG', ra))
        hdr.append(('DEC_TARG', dec))
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    start = np.min(starts)
    end = np.max(ends)
    
    for spec in seds:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        hdr.append(('MJD-BEG', start))
        hdr.append(('MJD-END', end))
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        for i in hdr:
            if hdr[i] in ['mod', 'Mod', 'MOD']:
                hdr[i] = 'MODEL'
        starname = hdr['TARGNAME']
        if starname[0:2].upper() == 'HD':
            starname = starname.replace('-', ' ')
        elif starname == 'kap1Cet':
            starname = '*kap01 Cet'
        simtab = Simbad.query_object(starname)
        coords = SkyCoord(simtab['RA'][0], simtab['DEC'][0], unit=(u.hourangle, u.deg))

        hdr['RA_TARG'] = coords.ra.deg
        hdr['DEC_TARG'] = coords.dec.deg
        
      
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)


toi-836
hd120411
hd80606
toi-178
lhs_1140
wasp-166
k2-18
gj_4102
kepler-51
k2-141
kappa1_ceti
wasp-63
toi-402
toi-776
kap1cet
toi-260
gj_341
tau_ceti
toi-421
gj_367
toi-134
wasp-121


In [6]:
146-66

80

In [7]:
# Simbad.query_objects(stars)

In [8]:
len(stars)

22

In [9]:
for i in hdr:
    if hdr[i] in ['mod', 'Mod', 'MOD']:
        print (i, hdr[i])
        hdr[i] = 'MODEL'
        print (i, hdr[i])

In [10]:
hdr

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
TELESCOP= 'MULTI   '                                                            
INSTRUME= 'MULTI   '                                                            
GRATING = 'MULTI   '                                                            
TELESC00= 'MODEL   '                                                            
INSTRU00= 'LYA     '                                                            
GRATIN00= 'YOUNG   '                                                            
TELESC01= 'HST     '                                                            
INSTRU01= 'STS     '                                                            
GRATIN01= 'G140L   '        

In [11]:
c = SkyCoord(star_list['RA'][-1], star_list['DEC'][-1], unit=(u.hourangle, u.deg))
c

<SkyCoord (ICRS): (ra, dec) in deg
    (107.60025167, -39.09738083)>

In [12]:
c.dec.deg


-39.09738083333333

In [13]:
for star in stars:
    print(star)
    starpath = 'draft_hlsp/{}/'.format(star)
    seds = glob.glob('{}*multi*.fits'.format(starpath))
    for sed in seds:
        hdul = fits.open(sed)
        hdr = hdul[0].header
        starname = hdr['TARGNAME']
        print(starname)
        if starname[0:2].upper() == 'HD':
            starname = starname.replace('-', ' ')
        elif starname == 'kap1Cet':
            starname = '*kap01 Cet'
        simtab = Simbad.query_object(starname)
        coords = SkyCoord(simtab['RA'][0], simtab['DEC'][0], unit=(u.hourangle, u.deg))
        print(coords.ra.deg, coords.dec.deg)
        print(hdr['RA_TARG'], hdr['DEC_TARG'])
        # print(startab)

toi-836
toi-836
225.08082624999997 -24.454081388888888
225.0794424294 -24.45425419529
toi-836
225.08082624999997 -24.454081388888888
225.0794424294 -24.45425419529
toi-836
225.08082624999997 -24.454081388888888
225.0794424294 -24.45425419529
toi-836
225.08082624999997 -24.454081388888888
225.0794424294 -24.45425419529
hd120411
hd-120411
207.52616541666666 -40.83580027777778
207.5259225153 -40.83594002641
hd-120411
207.52616541666666 -40.83580027777778
207.5259225153 -40.83594002641
hd-120411
207.52616541666666 -40.83580027777778
207.5259225153 -40.83594002641
hd-120411
207.52616541666666 -40.83580027777778
207.5259225153 -40.83594002641
hd80606
toi-178
lhs_1140
wasp-166
k2-18
K2-18
172.56049041666662 7.588404166666666
172.5599798266 7.587554876075
K2-18
172.56049041666662 7.588404166666666
172.5599798266 7.587554876075
K2-18
172.56049041666662 7.588404166666666
172.5599798266 7.587554876075
K2-18
172.56049041666662 7.588404166666666
172.5599798266 7.587554876075
gj_4102
kepler-51
keple

In [14]:
# simtab

In [15]:
# Simbad.query_object('TOI-134')